In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sb
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
labels = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for tracks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for track', 'Brock',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for track']

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                directory=r"./train/",
                                target_size=(40,40),
                                batch_size=32,
                                color_mode="rgb",
                                class_mode='categorical')


Found 39209 images belonging to 43 classes.


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(40, 40, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [7]:
model_log = model.fit_generator(train_generator,
                                steps_per_epoch=39209 // 64,
                                epochs=1)

Epoch 1/1
612/612 [==============================] - 479s 783ms/step - loss: 0.0231 - accuracy: 0.9931


In [8]:
traffic_json = model.to_json()
with open("traffic.json", "w") as json_file:
    json_file.write(traffic_json)
model.save_weights("traffic.h5")

In [39]:
import cv2
img = cv2.imread('26.png')
img=img/255.0
img=cv2.resize(img,(40,40))
img= np.expand_dims(img, axis =0)
p=model.predict(img)[0]

In [40]:
mx=0
for i in range(43):
    if p[i]>p[mx]:
        mx=i

In [41]:
print(labels[mx])

No overtaking for tracks


In [45]:
test=pd.read_csv('Test.csv')

In [49]:
y_test = test.iloc[:, 6].values

In [77]:
from PIL import Image
from numpy import asarray
image = cv2.imread('..\\Test\\00001.png')
image2 = asarray(image)

In [119]:
import cv2
import os

images = []
for filename in os.listdir("Test"):
    img = cv2.imread(os.path.join("Test",filename))
    img = img/255.0
    img = cv2.resize(img,(40,40))
    img = np.expand_dims(img, axis =0)
    if img is not None:
        images.append(img)

In [120]:
arr = np.array(images) 

In [121]:
arr.shape

(12630, 1, 40, 40, 3)

In [130]:
lis=[]
for i in range(12630):
    p=model.predict(arr[i])[0]
    mx=0
    for i in range(43):
        if p[i]>p[mx]:
            mx=i
    lis.append(mx)

In [131]:
y_pres = np.array(lis) 

In [137]:
y_pres.shape

(12630,)

In [134]:
y_test

array([16,  1, 38, ...,  6,  7, 10], dtype=int64)

In [139]:
c=0
for i in range(12630):
    if y_pres[i]==y_test[i]:
        c=c+1
print((c*100)/12630)

5.756136183689628
